# RayTune - Epochs, Learning Rate, Batch Size Tuning

### Imports

In [4]:
from functools import partial
import os
import random 

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split

from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.basic_variant import BasicVariantGenerator

In [5]:
torch.manual_seed(40)
random.seed(40)
np.random.seed(40)

In [6]:
# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

### Load Data

In [7]:
def load_data(data_dir="./data"):
    # transforms
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))])

    # datasets
    trainset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=True,
        transform=transform)
    testset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=False,
        transform=transform)
    return trainset, testset

In [9]:
trainset, testset = load_data()

100.0%


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100.0%


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100.0%


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100.0%

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



### Data Loaders

In [10]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [11]:
# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=4,
                                          shuffle=True, 
                                          num_workers=2,
                                          worker_init_fn=seed_worker,
                                          generator=g)


testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=2,
                                         worker_init_fn=seed_worker,
                                         generator=g)

### CNN Model Setup

In [12]:
# model definition
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Training

In [13]:
def train_fashion_mnist(config, data_dir=None):
    net = FashionCNN() 

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["learning_rate"])

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
    )

    for epoch in range(config["epochs"]):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1

            if i % 2000 == 1999:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / epoch_steps))
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_steps += 1

        tune.report(
            mean_accuracy=correct / total,
            mean_val_loss=val_loss / val_steps
        )

In [14]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [16]:
config = {
    "epochs": tune.choice([5, 10, 15, 20]),
    "learning_rate": tune.loguniform(1e-4, 1e-2), # 0.0001 to 0.01
    "batch_size": tune.choice([16, 32, 64, 128])
}

max_num_epochs = 20
num_samples = 10

# Async Hyperband Scheduler
scheduler = ASHAScheduler(
    metric="mean_val_loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2,
)

result = tune.run(
    partial(train_fashion_mnist, data_dir="./data"),
    resources_per_trial={"cpu": 8, "gpu": 0},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/',
    search_alg=BasicVariantGenerator(random_state=42),
    scheduler=scheduler)

2023-08-09 23:35:31,112	INFO worker.py:1621 -- Started a local Ray instance.
2023-08-09 23:35:35,291	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-08-09 23:35:35,295	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-08-09 23:35:35,297	WARNING syncer.py:260 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage provider, consider installing the respective fsspec-package (see https://github.com/fsspec).


(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz


0.4%(func pid=17616) 
0.5%(func pid=17616) 
1.4%(func pid=17616) 
1.5%(func pid=17616) 
2.5%(func pid=17616) 
4.1%(func pid=17616) 
5.2%(func pid=17616) 
5.3%(func pid=17616) 
5.5%(func pid=17616) 
6.2%(func pid=17616) 
6.3%(func pid=17616) 
7.6%(func pid=17616) 
8.8%(func pid=17616) 
10.0%(func pid=17616) 
11.7%(func pid=17616) 
12.8%(func pid=17616) 
13.0%(func pid=17616) 
14.1%(func pid=17616) 
14.3%(func pid=17616) 
14.5%(func pid=17616) 
15.8%(func pid=17616) 
15.9%(func pid=17616) 
17.0%(func pid=17616) 
18.1%(func pid=17616) 
18.2%(func pid=17616) 
19.3%(func pid=17616) 
19.6%(func pid=17616) 
20.8%(func pid=17616) 
22.1%(func pid=17616) 
23.3%(func pid=17616) 
23.4%(func pid=17616) 
23.6%(func pid=17616) 
23.7%(func pid=17616) 
24.8%(func pid=17616) 
24.9%(func pid=17616) 
26.0%(func pid=17616) 
26.3%(func pid=17616) 
27.7%(func pid=17616) 
29.0%(func pid=17616) 
30.3%(func pid=17616) 
30.4%(func pid=17616) 
30.5%(func pid=17616) 
31.7%(func pid=17616) 
33.0%(func pid=17616) 
3

(func pid=17616) Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw
(func pid=17616) 
(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz


100.0%36m(func pid=17616) 


...

100.0%36m(func pid=17616) 


(func pid=17616) Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw
(func pid=17616) 
(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz


16.3%(func pid=17616) 
17.0%(func pid=17616) 
21.5%(func pid=17616) 
48.2%(func pid=17616) 
49.6%(func pid=17616) 
54.1%(func pid=17616) 
83.0%(func pid=17616) 
84.5%(func pid=17616) 
86.7%(func pid=17616) 
87.4%(func pid=17616) 
88.9%(func pid=17616) 
91.1%(func pid=17616) 
92.6%(func pid=17616) 
94.1%(func pid=17616) 
100.0%36m(func pid=17616) 


(func pid=17616) Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw
(func pid=17616) 
(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
(func pid=17616) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz
(func pid=17616) Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw
(func pid=17616) 


100.0%36m(func pid=17616) 


In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")
print(f"Best trial config: {best_trial.config}")
print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")